# Импорт библиотек

In [2]:
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import numpy as np
import torchvision

from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import torch.onnx

# Функция загрузчика данных

Загрузчик данных с функцией разделения на трейн и валидацию.

In [3]:
data_dir = './data/classed_nrj_train/'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#torch.backends.cudnn.enabled = False

# Модели

## Функция модели `create_nn_v2`

In [40]:
#Объявляю функцию загрузчика с аргументами Директория, Размер батча, Размер валидационной выборки
def load_split_train_test_v2(datadir, batch_size, valid_size = .2): 
    #Трансформации для обучающей выборки
    train_transforms = transforms.Compose([transforms.CenterCrop(224), 
                                           transforms.RandomHorizontalFlip(p=0.5),
                                           transforms.RandomVerticalFlip(p=0.5),
                                           transforms.RandomRotation(degrees=(45, 90)),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    #Трансформации для тестовой выборки
    test_transforms = transforms.Compose([transforms.CenterCrop(224),
                                           transforms.ToTensor(),
                                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    #Загрузчик для тренировочной выборки
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    #Загрузчик для тренировочной выборки
    test_data = datasets.ImageFolder(datadir,
                    transform=test_transforms)
    #Код для разделения на трейн и тест в указанном соотношении
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=batch_size)
    return trainloader, testloader

In [41]:
#Получаю тренировочный и валидационный генераторы
train_loader, val_loader = load_split_train_test_v2(data_dir, 
                                                    batch_size=16,
                                                    valid_size = .2)
#Проверяю результаты работы генераторов
print(train_loader.dataset.classes)
print(val_loader.dataset.classes)
print(len(train_loader))
print(len(val_loader))

['1', '10', '20', '3', '30', '6']
['1', '10', '20', '3', '30', '6']
664
166


In [42]:
#Объявляю функцию для нейросети с аргументами Трей и тест загрузчиков,
#шагом обучения и количеством эпох
def create_nn_v2(train_loader,
                 test_loader,
                 learning_rate=0.01,
                 epochs=10
                 ):

    #Объявляю класс модели, определяю архитектуру
    class ConvNet(nn.Module): 
        def __init__(self):
            super(ConvNet, self).__init__()
            self.c1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
            self.c2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
            self.c3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
            self.fc1 = nn.Linear(in_features=6272, out_features=1024)
            self.fc2 = nn.Linear(in_features=1024, out_features=512)
            self.fc3 = nn.Linear(in_features=512, out_features=256)
            self.fc4 = nn.Linear(in_features=256, out_features=6)
        
        def forward(self, x):
            x = F.relu(F.max_pool2d(self.c1(x), 3))
            x = F.relu(F.max_pool2d(self.c2(x), 3))
            x = F.relu(F.max_pool2d(self.c3(x), 3))
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = self.fc4(x)

            return F.log_softmax(x,  dim=1)

    net = ConvNet()
    print(net)

    #Задаю функцию потерь
    criterion = nn.CrossEntropyLoss()
    #Задаю оптимизатор
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

    total_step = len(train_loader)
    loss_list = []
    acc_list = []
    for epoch in range(epochs):
        for i, (images, labels) in tqdm(enumerate(train_loader)):
            #Прямой запуск
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss_list.append(loss.item())

            #Обратное распространение и оптимизатор
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            #Отслеживание точности
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            acc_list.append(correct / total)

            if (i + 1) % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, epochs, i + 1, total_step, loss.item(),
                              (correct / total) * 100))
    #Проверка на валидационной выборке
    net.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in tqdm(test_loader):
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Test Accuracy of the model on the test images: {} %'
              .format((correct / total) * 100))

In [43]:
%%time
#Запускаю функцию нейросети
create_nn_v2(train_loader=train_loader,
             test_loader=val_loader,
             learning_rate=0.001,
             epochs=5)

0it [00:00, ?it/s]

ConvNet(
  (c1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (c2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (c3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=6272, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=6, bias=True)
)


100it [00:31,  3.29it/s]

Epoch [1/5], Step [100/664], Loss: 0.7981, Accuracy: 62.50%


200it [01:03,  3.20it/s]

...

Epoch [5/5], Step [500/664], Loss: 0.0190, Accuracy: 100.00%


600it [03:24,  2.98it/s]

Epoch [5/5], Step [600/664], Loss: 0.0007, Accuracy: 100.00%


664it [03:46,  2.93it/s]
100%|████████████████████████████████████████████████████████████████████████████████| 166/166 [00:25<00:00,  6.43it/s]

Test Accuracy of the model on the test images: 99.09638554216868 %
Wall time: 19min 5s
